In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] != "context":

    raise

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
for element_type, df in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    element_x_fit_parameter = pd.read_csv(
        PATH["{}_x_fit_parameter.tsv".format(element_type)], sep="\t", index_col=0
    )

    context = ccal.make_context_matrix(
        df,
        n_job=SETTING["n_job"],
        skew_t_pdf_fit_parameter=element_x_fit_parameter,
        output_file_path=PATH["feature_x_sample.{}_context.tsv".format(element_type)],
    )

    ccal.summarize_feature_x_sample(
        context,
        feature_x_sample_alias="{} Context".format(
            SETTING["{}_alias".format(element_type)]
        ),
        feature_x_sample_value_name="Context",
        plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
        plot_histogram_max_size=SETTING["plot_histogram_max_size"],
        plot_rug_max_size=SETTING["plot_rug_max_size"],
    )

    for context_type, clip_kwargs in (
        ("negative", {"upper": 0}),
        ("positive", {"lower": 0}),
    ):

        element_context = context.clip(**clip_kwargs).sum(axis=1).sort_values()

        if SETTING["{}s_to_peek".format(element_type)] is None:

            ranks = []

        else:

            ranks = np.nonzero(
                [
                    element in SETTING["{}s_to_peek".format(element_type)]
                    for element in element_context.index
                ]
            )[0]

        ccal.plot_points(
            (tuple(range(element_context.size)), ranks),
            (element_context, element_context[ranks]),
            names=("All", "Peek"),
            texts=(element_context.index, element_context.index[ranks]),
            modes=("markers", "markers+text"),
            title="{} Ranking by {} Context Sum".format(
                SETTING["{}_alias".format(element_type)], context_type.title()
            ),
            xaxis_title="Rank",
            yaxis_title="{} Context Sum".format(context_type.title()),
        )

        if element_context.unique().size < 2:

            continue

        n_extreme = 3

        if context_type == "negative":

            ranks = range(n_extreme)

        elif context_type == "positive":

            ranks = range(-n_extreme, 0)

        for rank in ranks:

            ranked_element = element_context.index[rank]

            ranked_element_values = df.loc[ranked_element]

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                ranked_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            ccal.plot_context(
                ranked_element_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=ranked_element_values.dropna().size
                < SETTING["plot_rug_max_size"],
                title="{} {} Context Sum Rank {}: {}".format(
                    SETTING["{}_alias".format(element_type)],
                    context_type.title(),
                    rank,
                    ranked_element,
                ),
                xaxis_title=SETTING["feature_x_sample_value_name"],
            )